# Data Reading

In [ ]:
import os
import sys
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

# Local modules
import nexrad

# iRadar
sys.path.append(os.path.expanduser('~/Developer/iradar/python'))
import iradar

In [ ]:
plt.rcParams['axes.facecolor'] = 'black'

In [ ]:
files = glob.glob(os.path.expanduser('~/Downloads/88d/KDDC/*V06'))

In [ ]:
# filename = os.path.expanduser('~/Downloads/88d/KDDC/KDDC20200131_010529_V06')
# filename = os.path.expanduser('~/Downloads/88d/KDDC/KDDC20200131_165722_V06')
filename = files[0]

azimuth_count = 360
range_count = 800

In [ ]:
sweeps = nexrad.read(filename)

m = sweeps['m'][0, :, :range_count]
z = sweeps['z'][0, :, :range_count]
v = sweeps['v'][0, :, :range_count]

m = np.logical_or(m, z < -5.0)
z[m] = np.nan
v[m] = np.nan

In [ ]:
# The overlay
overlay = iradar.atlas.Overlay((sweeps['longitude'], sweeps['latitude']))
overlay.load()
overlay.setRingRadii([1.0, 30.0, 60.0, 90.0])
overlay.prepareForRect([-50.0, -50.0, 100.0, 100.0])

# The chart
view = iradar.chart.Image(None, None, None, maxrange=80.0, figsize=(600, 675), y=+5, overlay=overlay)

# Data Viewing

In [ ]:
az = sweeps['aa'][0, :]
rr = 1.0e-3 * sweeps['rr'][:range_count]
view.set_data(np.zeros((azimuth_count, range_count)), a=az, r=rr, style='Z')

In [ ]:
im = []

view.set_data(z, style='Z')
im.append(view.pixel_buffer())

view.set_data(v, style='V')
im.append(view.pixel_buffer())

buff = np.array(np.hstack(im), dtype=np.uint8)
mosaic = iradar.chart.PIL.Image.fromarray(buff)

display(mosaic)

In [ ]:
# mosaic.save(os.path.expanduser('~/Downloads/88d/cache/{}.png'.format(os.path.basename(filename))))